# Magic Square Solver with Genetic Algorithm

In [ ]:
import numpy as np
import random

# Define constants
POPULATION_SIZE = 50
TARGET_SUM = 15
MAX_GENERATIONS = 1000
INITIAL_MUTATION_RATE = 0.2

def create_individual():
    """Create a random permutation of the numbers 1 to 9."""
    return np.random.permutation(9) + 1

def crossover(parent1, parent2):
    """Perform two-point crossover between two parents to produce a child."""
    crossover_points = sorted(random.sample(range(len(parent1)), 2))
    child = np.hstack((parent1[:crossover_points[0]], parent2[crossover_points[0]:crossover_points[1]], parent1[crossover_points[1]:]))
    return child

def mutate(individual, mutation_rate):
    """Mutate an individual by randomly swapping two numbers."""
    for _ in range(2):  # Mutate two random positions
        if random.random() < mutation_rate:
            idx1, idx2 = random.sample(range(len(individual)), 2)
            individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

def fitness(individual):
    """Calculate the fitness of an individual by computing the sum of rows, columns, and diagonals."""
    square = individual.reshape((3, 3))
    sums = [np.sum(square, axis=0),  # Column sums
            np.sum(square, axis=1),  # Row sums
            [np.trace(square), np.trace(np.fliplr(square))]]  # Diagonal sums
    # Calculate penalties for each sum
    penalties = [np.sum(np.abs(np.array(s) - TARGET_SUM)) for s in sums]
    total_penalty = np.sum(penalties)
    return 1 / (1 + total_penalty)  # Fitness is inversely proportional to penalty

def select_parents(population):
    """Select two parents based on their fitness using tournament selection."""
    tournament_size = 5
    selected_parents = []
    for _ in range(2):
        candidates = random.sample(population, tournament_size)
        selected_parents.append(max(candidates, key=fitness))
    return selected_parents

def genetic_algorithm():
    """Main genetic algorithm loop."""
    # Initialize population with some known patterns
    population = [create_individual() for _ in range(POPULATION_SIZE)]
    population.extend([
        np.array([1, 5, 9, 3, 5, 7, 2, 5, 8]),  # A known pattern close to solution
        np.array([9, 5, 1, 7, 5, 3, 8, 5, 2])   # Another known pattern close to solution
    ])

    mutation_rate = INITIAL_MUTATION_RATE

    # Main loop
    for generation in range(MAX_GENERATIONS):
        # Evaluate fitness of each individual
        fitness_scores = [fitness(individual) for individual in population]

        # Check for solution
        if 1 in fitness_scores:
            solution_idx = fitness_scores.index(1)
            print("Solution found in {} generations:".format(generation + 1))
            print(population[solution_idx].reshape((3, 3)))
            return

        # Select parents
        parents = [select_parents(population) for _ in range(POPULATION_SIZE // 2)]

        # Perform crossover and mutation
        offspring = []
        for parent1, parent2 in parents:
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            offspring.append(child)

        # Replace old population with offspring
        population = offspring

        # Update mutation rate
        mutation_rate = max(0.05, mutation_rate - 0.0001)

    print("No solution found within the maximum number of generations.")

if __name__ == "__main__":
    genetic_algorithm()